------
## 숙제

* 지난 번 숙제를 통해 다운 받은 이미지에 대한 메타 데이터를 추출하세요.  

* 추출된 메타 데이터에 대한 통계값을 계산하세요.  

* 추출된 메터 데이터는 별도 파일로 저장하고(json 또는 csv 포멧)

   계산된 통계값은 화면에 출력하세요 (최대한 이쁘게)
   
   
* 파일별로 추출할 메타 데이터는 다음과 같습니다.
  * 파일 이름
  * 상위 폴더명
  * 이미지 확장자 
  * 이미지 해상도  
    
* 계산할 통계값은 다음과 같습니다.
  * 이미지 해상도 통계
    * 이미지 해상도 종류
    * 각 이미지 해상도별 사진 갯수
    * 이미지 넓이(픽셀 높이 x 픽셀 넓이)의 대표값
        * 평균값, 최소값, 최대값, 최빈값, 중앙값
  * 이미지 확장자 통계
    * 확장자 종류
    * 각 이미지 확장자별 사진 갯수
  * 폴더별 통계
    * 각 폴더별 이미지 갯수
 
  
* 이미지 해상도 알아내는 방법은 아래 패키지를 이용하세요. 
  * https://github.com/shibukawa/imagesize_py


In [8]:
from pathlib  import Path
from collections import Counter
import json
import statistics
import pandas as pd
try: 
    import imagesize
except:
    !pip install imagesize
    import imagesize
    
# 폴더 내의 파일들에 대해 메타 정보 추출하기
def extract_metas(folder):
    def extract_meta(path):
        path = Path(path)
        return {
            'name': path.name,
            'parent': path.parent,
            'suffix': path.suffix,
            'size': imagesize.get(path)
        }
    return [extract_meta(f) for f in Path(folder).iterdir() if not f.is_dir()]

# 메타 정보 폴더별로 추출하기
metas = [extract_metas(d) for d in Path('downloads').iterdir() if d.is_dir()]

# 폴더별로 추출된 것 합치기: flatten
metas = sum(metas, [])

def valid(metas):
    l = []
    for meta in metas:
        sizes = meta['size']
        w, h = sizes
        if w > 0 and h > 0:
            l.append(meta)
    return l

metas = valid(metas)

# 메타 정보 통계값을 출력하세요.
def chunks(list, n):
    for i in range(0, len(list), n):
        yield list[i:i+n]
         
def print_statistics(metas):
    print('1. 이미지 해상도 통계')
    print('1-1. 이미지 해상도 종류는 뭐가 있을까?')
    pixels = [meta['size'] for meta in metas]
    pixels = Counter(pixels)
    keysList = list(pixels.keys())

    for i in range(0, len(keysList), 5):
        l = keysList[i:i+5]
        print(l, sep='\n')
    
    print('\n\n')
    print('1-2. 각 이미지 해상도별 개수를 알아보자')
    kvList = list(element for element in pixels.items())
    for i in range(0, len(kvList), 5):
        l_ = kvList[i:i+5]
        print(l_, sep='\n')
 
    print('\n\n')
    print('1-3. 각 이미지에 대한 넓이의 대표값구하기')
    sizes = [w*h for w,h in pixels]
    print('1-3-1 평균값: ', statistics.mean(sizes))
    print('1-3-2 최소값: ', min(sizes))
    print('1-3-3 최대값: ', max(sizes))
    print('1-3-4 최빗값: ', (lambda l: l.most_common(1))(pixels))
    print('1-3-5 중앙값: ', statistics.median(sizes))
    
    print('\n\n')
    print('2. 이미지 확장자 통계')
    print('2-1. 이미지 확장자 종류를 알아보자.')
    suffixes = [meta['suffix'] for meta in metas]
    suffixes = Counter(suffixes)
    keySuffix = list(suffixes.keys())
    
    for i in range(0, len(keySuffix), 5):
        l = keySuffix[i:i+5]
        print(l, sep='\n')
    
    print('\n\n')
    print('2-2. 각 이미지 확장자 별 개수를 알아봅시다')
    kvList = list(element for element in suffixes.items())
    print(kvList)
    
    print('\n\n')
    parents = []
    print('3. 폴더별 이미지 개수는?')
    for meta in metas:
        m = str(meta['parent']).split('\\')
        parent = m[-1]
        parents.append(parent)
    
    parentCnt = Counter(parents)
    for k,v in parentCnt.items():
        print(k, v)
    
# 메타 정보를 저장하는 함수를 작성하세요
def save_meta(metas, write_path):
    '''
    여기를 작성하세요.
    write_path 경로에 메타 데이터 리스트를 저장하세요
    json 또는 csv 포멧으로 저장하세요
    '''
    datas = pd.DataFrame(metas)
    datas.to_csv(write_path, header=False, index=False)
    
print_statistics(metas)
save_meta(metas, 'meta-data.csv')

1. 이미지 해상도 통계
1-1. 이미지 해상도 종류는 뭐가 있을까?
[(1440, 1024), (900, 660), (262, 376), (700, 700), (433, 268)]
[(631, 450), (640, 640), (682, 548), (800, 800), (1079, 698)]
[(700, 1561), (400, 400), (1280, 720), (600, 400), (596, 400)]
[(2100, 1190), (495, 442), (474, 438), (600, 600), (900, 490)]
[(680, 880), (591, 591), (424, 230), (551, 551), (640, 534)]
[(1450, 1000), (1160, 300), (391, 324), (500, 700), (558, 398)]
[(200, 200), (770, 430), (718, 904), (451, 541), (670, 670)]
[(756, 634), (494, 263), (400, 300), (500, 281), (2177, 824)]
[(680, 883), (743, 514), (638, 479), (640, 234), (474, 316)]
[(618, 536), (680, 440), (2352, 1314), (739, 593), (743, 907)]
[(794, 471), (568, 525), (1874, 1208), (750, 750), (760, 352)]
[(993, 252), (1600, 1131), (600, 424), (300, 225), (2356, 1408)]
[(620, 465), (800, 350), (1280, 800), (489, 232), (743, 443)]
[(1123, 794), (1100, 1500), (900, 900), (400, 500), (546, 260)]
[(1578, 1334), (800, 671), (500, 500), (540, 231), (773, 446)]
[(236, 218), (642, 38